In [14]:
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize
wpt = WordPunctTokenizer()
from string import punctuation
punctuation
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk import ngrams
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()
from tqdm import tqdm
from collections import Counter
import requests
import json
import time
import os
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
noise = stopwords.words('russian') + list(punctuation) + list(['``',"''",'...','!','--'])
from typing import Union,Optional
from datetime import datetime

trash = ['работа','компания','наш','компания','работать','год','который', 'график','новый','условие',
        'в','участие','возможность','•','умение','знание','понимание','–','—','свой','мы','задача',
         'заработный', 'плата','тк', 'рф','рабочий', 'день','требование', 'опыт','официальный', 
         'трудоустройство','оформление','высокий', 'образование','предстоять', 'заниматься','быть', 'плюс',
         'что', 'предлагать','чем', 'предстоять','требование','высокий','дружный', 'коллектив','профессиональный', 
         'рост','2', 'раз','что', 'нужно','раз', 'ум','стать','желание','грамотный', 'устный','момент','quot','ждать',
         '«', 'белый', '»','привет', 'я', 'звать','расти','интересный','достойный','вы','рада',
         'месяц','официальный', 'оформление','успешный', 'опыт','профессиональный', 'согласно','обязанность','не указано']

[nltk_data] Downloading package stopwords to C:\Users\Владислав
[nltk_data]     Дзюба\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def getAreas():
    req = requests.get('https://api.hh.ru/areas')    
    data = req.content.decode()
    req.close()
    jsObj = json.loads(data)
    areas = []
    for k in jsObj:
        for i in range(len(k['areas'])):
            if len(k['areas'][i]['areas']) != 0:                      # Если у зоны есть внутренние зоны
                for j in range(len(k['areas'][i]['areas'])):
                    areas.append([k['id'], 
                                  k['name'], 
                                  k['areas'][i]['areas'][j]['id'],
                                  k['areas'][i]['areas'][j]['name']])
            else:                                                                # Если у зоны нет внутренних зон
                areas.append([k['id'], 
                              k['name'], 
                              k['areas'][i]['id'], 
                              k['areas'][i]['name']])
     
    areas_dict = {}
    for i in areas:
        areas_dict[i[-1].lower()]=i[-2]
                
                
    return areas_dict

areas = getAreas()

In [4]:
### функция создания сравнительных датасэтов по n-граммам и стэку
def comparison_funk(df: pd.DataFrame,gramms: int=2,freq_amount: int=20) -> pd.DataFrame:
    
    df_gramms = pd.DataFrame()
    df_single_words = pd.DataFrame()
    df_stack = pd.DataFrame()
    print(df['experiences'].value_counts())
    print()
    
    # итерация по уникальным грейдам в конкретном датасэте
    for n,i in enumerate(df['experiences'].unique()):
        print(f'Анализируется категория: {i}')
        print('Работаем с описанием')
        
        ### обработка корпуса слов для биграмм и одиночных слов описания
        corpus = [pymorphy2_analyzer.parse(token)[0].normal_form for vacancy in df[df['experiences']==i]["description"] for token in word_tokenize(vacancy) if token not in noise]
        corpus = [word for word in corpus if word not in trash]
        
        ### выделение самых частотных n-грамм и слов описания
        freq_dict_single_words = Counter(corpus)
        single_words = freq_dict_single_words.most_common(freq_amount)
        
        freq_dict_ngram = Counter(ngrams(corpus, gramms))
        n_gramms = freq_dict_ngram.most_common(freq_amount)
            
        ### обработка корпуса ключевых слов 
        print('Работаем со стэком')
        corpus_stack = [token for vacancy in df[df['experiences']==i]['skills'] for token in vacancy.split(', ') if token not in noise]
        corpus_stack = [word for word in corpus_stack if word not in trash]
    
        ### выделение самых частотных слов в ключевых словах
        freq_dict_stack = Counter(corpus_stack)
        stack_words = freq_dict_stack.most_common(freq_amount)
        
        ### итеративное пополнение датасэтов по грейдам
        df_gramms[i]=n_gramms
        df_single_words[i] = single_words
        df_stack[i] = stack_words

        print('---------------------------------------')
    print('ГОТОВО')
    return df_gramms[['junior','1-3 years','3-6 years','6 years']],\
           df_single_words[['junior','1-3 years','3-6 years','6 years']],\
           df_stack[['junior','1-3 years','3-6 years','6 years']]

In [73]:

def get_vacancys(count: int=100, city: Optional[str] = None, name: Optional[str]=None,
                 search_fields: Union[str,list[str]] = ['company_name','name','description'],
                 experience: Union[str,list[str]] = ["noExperience",
                           "between1And3",
                           "between3And6", 
                           "moreThan6"])-> pd.DataFrame:
    
    ### на один и тот же запрос допустимо лишь 2000 вакансий, поэтому ограничиваем
    if count > 2000:
        print('Максимальное значение count на категорию 2000. Ваш count будет заменен на 2000')
        count=2000
        
    # если введен поиск только по одной категории, ее нужно перевести в список, для дальнейшего итерирования
    if type(experience) is str:
        experience = [experience]
        
    df = pd.DataFrame(columns=['name','skills','experiences','employments',
                               'professional_roles','salary','city','schedules','description','url'])
    
    
    ########################################################### META
    # общий накопительный счет собранных вакансий
    vacancys_amount = 0
    
    # отображение параметров запроса    
    print('Параметры запроса:')  
    print()
    print(f'Специальность: {name}')   
    
    if city is None:
        print('Город: любой')
    else:
        print(f'Город: {city}')
        
    date = str(datetime.now().date())
    meta_df = pd.DataFrame({'name': f'Запрос: {name}', 'skills': 'default', 'experiences': [experience],
                      'employments':'any','professional_roles':'any','salary':'any','city':city,
                      'schedules':'any','description':f'Поисковые поля: {search_fields}', 'url': f'Дата сбора: {date}'})
    df = pd.concat([df,meta_df])   

    # перевод названия города в индекс города, который необходим для запроса API
    if city is not None:
        areas = getAreas()
        try:
            city = areas[city.lower()]
        except:
            print('-----------------------------------------------------------------------------------------------')
            print('Ошибка в названии города или такого города не существует. Поиск продолжится по всем городам.')
            print('-----------------------------------------------------------------------------------------------')
            city = None

    print(f'Опыт: {experience}')
    print(f'Поисковые поля: {search_fields}')
    print('-----------------------------------------------')
    print()

    # начало итераций по спсику категорий опыта 
    for grade in experience: 
        print(f'Запрос категории опыта: {grade}')
        # инициализация спиков для заполнения полей из ответа API
        list_of_id = []
                     
        list_of_skills = []
        descriptions = [ ]
        names = [] 
        schedules = []
        employments = []
        experiences = []
        professional_roles = []
        salarys = []
        citys = []
        url = []
    
    
        # итерации по страницам, так как вырано количество на одну страницу 100 вакансий
        # запрошенное количество делится на 100 - столько кгругов пробежит цикл с 100 ваканциями на страницу
        for i in range(int(count/100)):
            # отображение номера обрабатываемой страниницы
            if i % 2 == 0:
                print(f'Запрос страницы: {i}')
            
            
            page = i
            params = {
                    'text': f'NAME:{name}', 
                    'areas': 113, #113 это Россия
                    'area': city, 
                    'page': page, # Индекс страницы поиска на HH
                    'per_page': 100, # вакансий на 1 странице
                    'search_field' : search_fields,
                    'experience': grade
                }
                
            # запрос к общиму поиску вакансий
            req = requests.get('https://api.hh.ru/vacancies', params) # запрос к API
            response = req.content.decode() # декодируем  ответ, чтобы кириллица отображалась корректно
            req.close()
            
            # получение спика id каждой вакансии
            list_of_id = [i['id'] for i in json.loads(response)['items']]
            
            if len(list_of_id)==0:
                break
    
            time.sleep(0.3)
            # итерация по спсиску id и получение подробной информации по каждой вакансии
            for step, id_ in enumerate(list_of_id):
                vacancy_req = requests.get(f'https://api.hh.ru/vacancies/{id_}')
                vacancy = vacancy_req.content.decode()
                result = json.loads(vacancy)
                
                url.append(result.get('alternate_url','Не указано'))
                list_of_skills.append([skill['name'] for skill in result.get('key_skills','не указано')])
                descriptions.append(result.get('description', 'не указано'))
                names.append(result.get('name','не указано')) 
                
                schedules.append((result.get('schedule',{}) or {}).get('name','не указано'))
                employments.append((result.get('employment',{}) or {}).get('name','не указано'))
                professional_roles.append((result.get('professional_roles',{}) or [{}]) [0].get('name','не указано'))

                experiences.append((result.get('experience','не указано') or {}).get('name','не указано'))
                citys.append((result.get('area','не указано') or {}).get('name','не указано'))
                
                salary = result.get('salary',{}) or "не указано"
                if salary != "не указано":
                    salary=str((salary.get('from','не указано') or 'не указано'))+' - '+str((salary.get('to','не указано') or 'не указано')) 
                salarys.append(salary)
                time.sleep(0.3) 
                
            # итеративное пополнение счетчика вакансий    
            vacancys_amount +=len(list_of_id)  
            if i % 2 == 0:
                print('Собрано вакансий:',vacancys_amount)
                print('-------------------------------------')
        
                   
        # создание датасэта н акаждой итерации          
        df_on_iter = pd.DataFrame({'name': names,'skills':list_of_skills,
                           'experiences':experiences,'employments':employments,'professional_roles':professional_roles,
                          'salary':salarys,'city':citys, 'schedules': schedules, 'description': descriptions,'url':url})
        
        ### чистка описания 
        df_on_iter['description'] = df_on_iter['description'].apply(lambda x: re.sub(r'<\/?\w+ ?\/?>',' ',x)).\
                                        apply(lambda b: re.sub(r'^\s+|\s+$', '',b)).\
                                        apply(lambda b: re.sub(r'( )(\1)+',r'\1',b))
        
        ### в key_skills может возвращаться пустой список, если не указаны навыки
        ### заменяем на "не указано"
        df_on_iter['skills'] = df_on_iter['skills'].apply(lambda x: ', '.join(x).lower()).\
                                    apply(lambda s: 'не указано' if len(s) == 0 else s)
        
#       добавление его к общему датасэту  
        df = pd.concat([df,df_on_iter],axis=0,ignore_index=True)
        
    # перевод опыта в более привычный вид    
    df.loc[df['experiences']=='Нет опыта','experiences']='junior'
    df.loc[df['experiences']=='От 1 года до 3 лет','experiences']='1-3 years'
    df.loc[df['experiences']=='От 3 до 6 лет','experiences']='3-6 years'
    df.loc[df['experiences']=='Более 6 лет','experiences']='6 years'
    print()
    print('-------------------------------------------')
    print('ЗАВЕРШЕНО:')
    print(f'Всего собрано вакансий: {vacancys_amount}')
    df.rename(index={0: 'MetaData'},inplace=True)
    return df

In [74]:
# experience = [ "noExperience", "between1And3","between3And6", "moreThan6"]
# search_fields = ['company_name','name','description']

data_search_in_name = get_vacancys(100,name='Маркетолог',search_fields='name',experience='noExperience')

Параметры запроса:

Специальность: Маркетолог
Город: любой
Опыт: ['noExperience']
Поисковые поля: name
-----------------------------------------------

Запрос категории опыта: noExperience
Запрос страницы: 0
Собрано вакансий: 100
-------------------------------------

-------------------------------------------
ЗАВЕРШЕНО:
Всего собрано вакансий: 100


In [77]:
data_search_in_name.loc[1:10]

,name,skills,experiences,employments,professional_roles,salary,city,schedules,description,url
1,Маркетолог,"медиапланирование, запуск региональных рекламн...",junior,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",60000 - 80000,Ростов-на-Дону,Полный день,&quot;Дентал Менеджмент&quot; управляет четырь...,https://hh.ru/vacancy/86345251
2,Младший специалист по маркетингу,не указано,junior,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",не указано,Москва,Полный день,В настоящий момент у нас открыта вакансия – Ме...,https://hh.ru/vacancy/87278617
3,Маркетолог / Помощник руководителя отдела марк...,"реклама, контекстная реклама, seo (на начально...",junior,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",60000 - 80000,Москва,Полный день,ООО &quot;СМАРТКОМПЛЕКТ&quot; – это сервисная ...,https://hh.ru/vacancy/86539406
4,Маркетолог,"digital marketing, навыки презентации, маркети...",junior,Стажировка,"Менеджер по маркетингу, интернет-маркетолог",500000 - 1000000,Алматы,Полный день,"Обязанности: Стратегия, тактика и план Анализ ...",https://hh.ru/vacancy/87181487
5,Помощник Маркетолога SMM,"smm, социальные сети, написание текстов, испол...",junior,Полная занятость,"SMM-менеджер, контент-менеджер",250000 - 300000,Алматы,Полный день,Обязанности: Грамотная устная и письменная реч...,https://hh.ru/vacancy/86619046
6,Специалист отдела маркетинга и рекламы,не указано,junior,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",1500 - не указано,Минск,Полный день,СОАО &quot;Коммунарка&quot; приглашает на рабо...,https://hh.ru/vacancy/82051817
7,Помощник маркетолога,"интернет-реклама, таргетинг, написание рекламн...",junior,Частичная занятость,"Менеджер по маркетингу, интернет-маркетолог",700 - 1800,Минск,Гибкий график,ВНИМАНИЕ! Ищем помощника маркетолога! В команд...,https://hh.ru/vacancy/87308699
8,Ассистент маркетолога,"контекстная реклама, копирайтинг, интернет-рек...",junior,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",150000 - 250000,Алматы,Удаленная работа,Привет! Мы — команда драйвовых опытных маркето...,https://hh.ru/vacancy/87126677
9,Ассистент маркетолога,не указано,junior,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",43800 - не указано,Москва,Полный день,Мы - арбитражная команда ZenLeads! Наша команд...,https://hh.ru/vacancy/86057715
10,Помощник маркетолога / Контент-менеджер,"cоциальный маркетинг, копирайтинг, smm, админи...",junior,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",800 - не указано,Минск,Полный день,Группа компаний ООО”НЭФОКС ПЛЮС” на протяжении...,https://hh.ru/vacancy/87269082


In [14]:
page = 1
params = {
        'text': f'NAME:Марктолог', 
        'areas': 113, #113 это Россия
        'area': 1, 
        'page': page, # Индекс страницы поиска на HH
        'per_page': 100, # вакансий на 1 странице
    }
    
# запрос к общиму поиску вакансий
req = requests.get('https://api.hh.ru/vacancies/86914042') # запрос к API
response = req.content.decode() # декодируем  ответ, чтобы кириллица отображалась корректно
req.close()
h = json.loads(response)

# Эксперимент

In [63]:
df_for_recomendate = pd.concat([marketologs_in_name,ds_in_name,cmo_in_name,developers_in_name])

In [64]:
df_for_recomendate['name_for_parse'] = df_for_recomendate['name'].\
                                apply(lambda x: ' '.join([pymorphy2_analyzer.parse(token)[0].normal_form \
                                                 for token in word_tokenize(x) if token not in noise]))
df_for_recomendate['name_for_parse'] 

0             менеджер маркетинг продвижение бренд
1             помощник маркетолог удалённый работа
2                             ассистент маркетолог
3                       маркетолог-стажёр помощник
4                         junior marketing manager
                           ...                    
1277             php/go golang разработчик backend
1278             php/go golang разработчик backend
1279                  backend php developer senior
1280             php/go golang разработчик backend
1281    старший бэкенд-разработчик scala вертикаль
Name: name_for_parse, Length: 7012, dtype: object

In [66]:
tf = TfidfVectorizer()
vectors = tf.fit_transform(df_for_recomendate['name_for_parse'])

neigh = NearestNeighbors(n_neighbors=10, n_jobs=-1, metric='euclidean')
neigh.fit(vectors)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=10)

In [84]:
marketologs_in_name

,name,skills,employments,professional_roles,salary,city,schedules,description,url,experiences
0,Менеджер по маркетингу и продвижению бренда,не указано,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",100000-170000,Москва,Полный день,Обязанности: сопровождение и развитие действую...,https://hh.ru/vacancy/87019437,junior
1,Помощник маркетолога (удаленная работа),"интернет-реклама, internet marketing, яндекс.д...",Частичная занятость,"Менеджер по маркетингу, интернет-маркетолог",19000-25000,Москва,Удаленная работа,"Привет! Меня зовут Дмитрий, я владелец консалт...",https://hh.ru/vacancy/85964259,junior
2,Ассистент маркетолога,"ms excel, маркетинг, планирование маркетинговы...",Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",None-80000,Москва,Полный день,Обязанности: запуск и контроль проведения трей...,https://hh.ru/vacancy/86974695,junior
3,Маркетолог-стажер помощник,"исполнение личных поручений руководителя, испо...",Стажировка,"Секретарь, помощник руководителя, ассистент",100000-200000,Алматы,Полный день,"Мы, рекрутинговое агентство RedLAb, занимаемся...",https://hh.ru/vacancy/86484189,junior
4,Junior marketing manager,"adobe photoshop, adobe illustrator, креативнос...",Полная занятость,Другое,60000-None,Москва,Полный день,Активно набирающая компания Happy Trade в поис...,https://hh.ru/vacancy/86180835,junior
...,...,...,...,...,...,...,...,...,...,...
4883,Трейд маркетолог,"маркетинговый анализ, анализ конкурентной сред...",Полная занятость,Руководитель отдела маркетинга и рекламы,8000000-12000000,Ташкент,Полный день,Мы - ведущая компания в сфере производства выс...,https://hh.ru/vacancy/85639866,6 years
4884,Маркетолог,не указано,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Не указано,Алматы,Полный день,"В группу компании, которая реализует услуги в ...",https://hh.ru/vacancy/85397805,6 years
4885,Маркетолог (развитие и продвижение продуктов У...,не указано,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Не указано,Москва,Полный день,Обязанности: Основная задача – развитие бренда...,https://hh.ru/vacancy/85640726,6 years
4886,Директор по маркетингу/Ведущий маркетолог,"продвижение бренда, маркетинговая стратегия, м...",Полная занятость,Руководитель отдела маркетинга и рекламы,Не указано,Санкт-Петербург,Полный день,На сегодня архитектурно-строительная компания ...,https://hh.ru/vacancy/85576389,6 years


In [82]:
role = 'Секретарь, помощник руководителя, ассистент'
normalazed_roles = [pymorphy2_analyzer.parse(token)[0].normal_form for token in word_tokenize(role) if token not in noise]
normalazed_roles =  tf.transform(normalazed_roles)

In [83]:
res = neigh.kneighbors(normalazed_roles, return_distance=True)
for i in res[1][0]:
  print(df_for_recomendate.iloc[i]['name'])

Директор по маркетингу (CMO/Chief Marketing Officer)
Директор по маркетингу / Chief Marketing Officer (CMO)
Аналитик-маркетолог (полная/частичная занятость, удаленно)
Маркетолог / интернет-маркетолог
Маркетолог/интернет-маркетолог
Руководитель отдела маркетинга/Интернет маркетолог в онлайн школу (удаленно)
Маркетолог / Интернет-маркетолог
Руководитель отдела маркетинга (главный маркетолог)
Менеджер SMM с обязанностями маркетолога
Senior NLP Data Scientist (команда SberCare.Боты)


In [47]:

corpus = [pymorphy2_analyzer.parse(token)[0].normal_form for vacancy in developers[developers['Опыт']=='1-3 years']["Описание"] for token in word_tokenize(vacancy) if token not in noise]

corpus = [word for word in corpus if word not in trash]
print(len(corpus))

freq_dict = Counter(corpus)

freq_dict.most_common(10)

69987


[('разработка', 1263),
 ('команда', 875),
 ('проект', 731),
 ('система', 527),
 ('код', 432),
 ('сервис', 401),
 ('продукт', 399),
 ('разработчик', 375),
 ('офис', 353),
 ('api', 345)]

In [26]:
freq_dict = Counter(ngrams(corpus, 2))

years_3_6 = freq_dict.most_common(30)